In [1]:
!pip install pandas
!pip install pickle-mixin
!pip install rowordnet
!pip install matplotlib

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6008 sha256=06376d90256e87966e8aefb9164857443948a1bca88f57e80d5b98eae5c090ca
  Stored in directory: /root/.cache/pip/wheels/d0/70/0b/673e09a7ed429660d22352a1b117b4f616a8fc054bdd7eb157
Successfully built pickle-mixin
     |████████████████████████████████| 11.8 MB 4.7 MB/s 


In [2]:
import pandas as pd
import pickle
import rowordnet as rwn
import matplotlib.pyplot as plt

In [4]:
!gdown --id 1IV_nodlm-dw-EWl1DtngkATgAldEdAGO # download pickle
!gdown --id 15xj8tQ9DF1B8ZZCK9KfSKnhvDydwfQTk
!gdown --id 16aH8Y05OD9uk6zek8kTKQZS21SDopm30
!gdown --id 16f2aGbV_MlR2QlgosD7C4saZIb1TCsLf
!gdown --id 16T1Ebt9rEwzpFXZPKLt_0zdkA23_zQ-m
!gdown --id 16HmaDk5NJYibrvjApTGPCAWsL6ClGttl
!gdown --id 16nGo42kKP9JrcpyyWzulf6TeHbIuMAYz
!gdown --id 16tjPu73ctm3MJXCdMb1GAkkec-H5bmkZ
!gdown --id 1610S7q5nsINwlYmiggZFumQtC4Mtr656
!gdown --id 172orWfZcMAexjDPgYGPWtFZmoJjnJegG
!gdown --id 15sbcdFGZlRPys2ljvrO-N9xmYMqGwmdh
!gdown --id 16tz-5yFCk09qaLC6vPHEVAoUEuBKc067
!gdown --id 15xuGMJyDMJng8S4DSODPna8SgIZfgpHu
!gdown --id 1592TZuICOat_VV7CqiwXjcDyjdFmB7cs
!gdown --id 16NdU_4nXMvSIa3zYtEkwWlrKH0g6FLJv

Downloading...
From: https://drive.google.com/uc?id=1IV_nodlm-dw-EWl1DtngkATgAldEdAGO
To: /content/dataset.pickle
100% 93.3M/93.3M [00:00<00:00, 200MB/s]
Downloading...
From: https://drive.google.com/uc?id=15xj8tQ9DF1B8ZZCK9KfSKnhvDydwfQTk
To: /content/bert.csv
100% 457k/457k [00:00<00:00, 64.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=16aH8Y05OD9uk6zek8kTKQZS21SDopm30
To: /content/bert.txt
100% 3.79k/3.79k [00:00<00:00, 3.31MB/s]
Downloading...
From: https://drive.google.com/uc?id=16f2aGbV_MlR2QlgosD7C4saZIb1TCsLf
To: /content/most_likely_sense_loose.txt
100% 212k/212k [00:00<00:00, 76.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=16T1Ebt9rEwzpFXZPKLt_0zdkA23_zQ-m
To: /content/most_likely_sense_strict.txt
100% 215k/215k [00:00<00:00, 65.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=16HmaDk5NJYibrvjApTGPCAWsL6ClGttl
To: /content/most_likely_sense.csv
100% 9.47M/9.47M [00:00<00:00, 55.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=16n

In [6]:
wn = rwn.RoWordNet()

def create_word_statistics(word):
    most_likely = pd.read_csv("most_likely_sense.csv")
    simplified = pd.read_csv("simplified_lesk.csv")
    simplified_wordnet = pd.read_csv("simplified_lesk_wordnet.csv")
    simplified_wordnet_complete = pd.read_csv("simplified_lesk_wordnet_complete.csv")
    bert = pd.read_csv("bert.csv")

    most_likely.loc[:, 'Algorithm'] = "Most Likely Sense"
    simplified.loc[:, 'Algorithm'] = "Simplified Lesk"
    simplified_wordnet.loc[:, 'Algorithm'] = "Simplified Lesk Wordnet"
    simplified_wordnet_complete.loc[:, 'Algorithm'] = "Simplified Lesk Wordnet Complete"
    bert.loc[:, 'Algorithm'] = "BERT"

    word_data = most_likely[most_likely['word'] == word].head(1)
    simplified = simplified[simplified['word'] == word].head(1)
    simplified_wordnet = simplified_wordnet[simplified_wordnet['word'] == word].head(1)
    simplified_wordnet_complete = simplified_wordnet_complete[simplified_wordnet_complete['word'] == word].head(1)
    bert = bert[bert['word'] == word].head(1)

    word_data = pd.concat([word_data, simplified])
    word_data = pd.concat([word_data, simplified_wordnet])
    word_data = pd.concat([word_data, simplified_wordnet_complete])
    word_data = pd.concat([word_data, bert])

    return word_data

def get_definitions_for_word(word):
    with open("dataset.pickle", "rb") as pickleFile:
        db = pickle.load(pickleFile)
        synsets = db[word][0]['synsets'].split()
        definitions = []
        for syn in synsets:
            try:
                definitions.append(wn.synset(syn).definition)
            except:
                continue
        
        correct_synset_id = db[word][0]['correct_synset_id']
        correct_definition = wn.synset(correct_synset_id).definition

    return definitions, correct_definition

def print_algorithm_statistics(alg, word_data):
    print("----Algorithm "+ alg + "----")
    alg_data = word_data[word_data["Algorithm"] == alg]

    predicted_synsets = alg_data['predicted_synsets'].to_list()[0].split()

    if (len(predicted_synsets) > 1):
        print("Synsets with maximum score")

        for syn in predicted_synsets:
                try:
                    print("* " + wn.synset(syn).definition)
                except:
                    continue

    print()
    print("Predicted definition")
    chosen_synset = alg_data['chosen_synset'].to_list()[0]
    print(wn.synset(chosen_synset).definition)
    print()

def main(word):
    print("Chosen word: " + word)
    word_data = create_word_statistics(word)
    sentence_data = word_data[word_data["Algorithm"] == "BERT"]
    sentence = sentence_data['sentence_id'].to_list()[0]
    definitions, marked_as_correct = get_definitions_for_word(word)

    print("Sentence: ")
    print(sentence)
    print()

    print("All possible definitions: ")
    for definition in definitions:
        print("* " + definition)
    print()

    print("Marked as correct: ")
    print(marked_as_correct)
    print()

    print_algorithm_statistics("Simplified Lesk", word_data)
    print_algorithm_statistics("Most Likely Sense", word_data)
    print_algorithm_statistics("Simplified Lesk Wordnet", word_data)
    print_algorithm_statistics("Simplified Lesk Wordnet Complete", word_data)
    print_algorithm_statistics("BERT", word_data)
    return


if __name__ == "__main__":
    main("familie")

Chosen word: familie
Sentence: 
În zona Esperance prima familie care s-a stabilit a fost familia Dempsters, o familie pionier de origine scoțiană, în anii 1870.

All possible definitions: 
* (biol) Categorie sistematică în botanică și în zoologie, inferioară ordinului, care cuprinde mai multe genuri de organisme cu caractere comune.;
* Formă socială de bază, întemeiată prin căsătorie, și care constă din soț, soție și din descendenții acestora
* Totalitatea celor care locuiesc împreună, (formând o familie
* familie aristocrată
* Totalitatea persoanelor care se trag dintr-un strămoș comun (rude de sânge); neam, descendență.
* Grup larg de oameni cu interese, idealuri și activități comune.
* Totalitatea persoanelor dintr-o linie de familie
* familia, neamul din care se trage cineva

Marked as correct: 
Formă socială de bază, întemeiată prin căsătorie, și care constă din soț, soție și din descendenții acestora

----Algorithm Simplified Lesk----
Synsets with maximum score
* (biol) Categorie